In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a regression problem.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [ ]:
# third-party libraries
import pandas as pd
import numpy as np
import os

import time
from tqdm.notebook import tqdm

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# source code
from src.utils import load_data, load_config

In [ ]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [ ]:
np.random.seed(42)

# Paths

In [ ]:
os.chdir("../") # set working directory to root of project
#os.getcwd() # check current working directory

In [ ]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [ ]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [ ]:
train_data, test_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1, raw=False)

2024-05-09 17:28:07 [src.utils:58] [INFO] >>>> Loading data set 1...
2024-05-09 17:28:08 [src.utils:100] [INFO] >>>> Loaded feature and target data for dataset 1.
2024-05-09 17:28:08 [src.utils:101] [INFO] >>>> Train Data: (20631, 27)
2024-05-09 17:28:08 [src.utils:102] [INFO] >>>> Test Data: (13096, 27)


# 📍 Subtask 2: Sliding Window Method

Findings:
* Interpretation of plots
* or other key take aways from previous code

In [12]:
# save processed data (as pickle)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_pickle(f"{config['paths']['processed_data_dir']}ex2_topic_{timestamp}.pkl")

In [ ]:
# save data predictions (as csv)
df = pd.DataFrame()
timestamp = time.strftime("%Y%m%d-%H%M%S")
df.to_csv(f"{config['paths']['prediction_dir']}ex2_topic_{timestamp}.csv", sep=',', decimal='.')

In [ ]:
# save plot results (as png)
fig = plt.figure(figsize=(9, 6))
timestamp = time.strftime("%Y%m%d-%H%M%S")
fig.savefig(f"{config['paths']['plot_dir']}ex2_topic_{timestamp}.png")